In [1]:
import os 
import random
import numpy as np

import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

In [2]:
main_dir = "/Users/devin/Documents/FYP/HAR-ZSL-XAI"
data_dir = os.path.join(main_dir,"sequence_data")
class_names = os.listdir(data_dir)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 1-train_ratio - val_ratio
batch_size = 32

In [3]:
def classname_id(class_name_list):
    id2classname = {k:v for k, v in zip(list(range(len(class_name_list))),class_name_list)}
    classname2id = {v:k for k, v in id2classname.items()}
    return id2classname, classname2id

In [4]:
class_names = os.listdir(os.path.join(main_dir,"data","pose_data"))

In [5]:
id2clsname, clsname2id = classname_id(class_names)

In [6]:
train_file_list = []
val_file_list = []
test_file_list = []

file_list = [os.path.join(data_dir,x) for x in os.listdir(data_dir)]

random.shuffle(file_list)
num_list = len(file_list)

train_range = [0,int(num_list*train_ratio)]
val_range = [int(num_list*train_ratio),int(num_list*(train_ratio+val_ratio))]
test_range = [int(num_list*(train_ratio+val_ratio)),num_list-1]

train_file_list += file_list[train_range[0]:train_range[1]]
val_file_list += file_list[val_range[0]:val_range[1]]
test_file_list += file_list[test_range[0]:test_range[1]]

In [7]:
len(train_file_list),len(val_file_list),len(test_file_list)

(12357, 2648, 2647)

In [8]:
train_file_list = train_file_list[:(len(train_file_list)//batch_size)*batch_size]
val_file_list = val_file_list[:(len(val_file_list)//batch_size)*batch_size]
test_file_list = test_file_list[:(len(test_file_list)//batch_size)*batch_size]

In [9]:
len(train_file_list),len(val_file_list),len(test_file_list)

(12352, 2624, 2624)

In [10]:
class SkeletonDataset(Dataset):
    def __init__(self, file_list,class2id, transform=None, 
                 target_transform=None,active_locations=[11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]):
        self.file_list = file_list
        self.transform = transform
        self.class2id = class2id
        self.target_transform = target_transform
        self.active_locations = active_locations

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        a_file = np.load(self.file_list[idx])
        action_type = self.file_list[idx].strip().split("/")[-1].split("_cls_")[0]
        coords, vid_size = a_file["coords"],a_file["video_size"]
        coords = coords[:,self.active_locations,:]
        
        shape = coords.shape
        
        coords = torch.from_numpy(coords).float()
        
        coords = torch.reshape(coords, (shape[0], shape[1]*shape[2]))
        label = torch.clone(coords)
        
        if self.transform:
            coords = self.transform(coords)
        if self.target_transform:
            label = self.target_transform(coords)
        return coords, label, self.class2id[action_type]

In [11]:
train_data = SkeletonDataset(train_file_list,clsname2id)
val_data = SkeletonDataset(val_file_list,clsname2id)
test_data = SkeletonDataset(test_file_list,clsname2id)

In [12]:
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=False)
val_dl = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [13]:
from tcn import TCN

class TCNEnc(nn.Module):
    def __init__(self,seq_len = 50,ts_dimension = 1,batch_size = 32,dilations = (1, 2, 4, 8, 16),nb_filters = 20,kernel_size = 20,nb_stacks = 1,padding = 'same',dropout_rate = 0.00,filters_conv1d = 8,pooler = nn.AvgPool1d,sampling_factor = 2,):
        super(TCNEnc, self).__init__()
        """
        Parameters
        ----------
        ts_dimension : int
            The dimension of the time series (default is 1)
        dilations : tuple
            The dilation rates used in the TCN-AE model (default is (1, 2, 4, 8, 16))
        nb_filters : int
            The number of filters used in the dilated convolutional layers. All dilated conv. layers use the same number of filters (default is 20)
        """
        
        self.seq_len = seq_len
        self.ts_dimension = ts_dimension
        self.dilations = dilations
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.nb_stacks = nb_stacks
        self.padding = padding
        self.dropout_rate = dropout_rate
        self.filters_conv1d = filters_conv1d
        self.pooler = pooler
        self.sampling_factor = sampling_factor
        self.batch_size = batch_size
        
        # build the model
        self.build_model()
        
    def build_model(self):
        """Builds the TCN-AE model.
        If the argument `verbose` isn't passed in, the default verbosity level is used.
        Parameters
        ----------
        verbose : str, optional
            The verbosity level (default is 1)
            
        Returns
        -------
        KerasXYZType
        Todo
        Raises
        ------
        NotImplementedError
            If ...
        """

        # Put signal through TCN. Output-shape: (batch, sequence length, nb_filters)
        self.tcn_enc = TCN(50,nb_filters=self.nb_filters, kernel_size=self.kernel_size, nb_stacks=self.nb_stacks, dilations=self.dilations, 
                      padding=self.padding, use_skip_connections=True, dropout_rate=self.dropout_rate, return_sequences=True)
        
        
        self.input_conv1d = self.nb_filters[-1] if isinstance(self.nb_filters,list) else self.nb_filters
        # Now, adjust the number of channels...
        self.enc_flat_conv = nn.Conv1d(self.input_conv1d, self.filters_conv1d, kernel_size=1, padding=self.padding)

        ## Do some average (max) pooling to get a compressed representation of the time series (e.g. a sequence of length 8)
        self.enc_pooled = self.pooler(self.sampling_factor)
        
        # If you want, maybe put the pooled values through a non-linear Activation
        self.enc_out = nn.ReLU()
        
    def forward(self,x):
        x = self.tcn_enc(x)
        x = self.enc_flat_conv(x)
        x = self.enc_pooled(x)
        
        #flatten 
        x= torch.flatten(x, start_dim=1)
        x = self.enc_out(x)
        
        return x
        
class TCNDec(nn.Module):
    def __init__(self,seq_len = 50,ts_dimension = 1,batch_size = 32,dilations = (1, 2, 4, 8, 16),nb_filters = 20,kernel_size = 20,nb_stacks = 1,padding = 'same',dropout_rate = 0.00,filters_conv1d = 8,pooler = nn.AvgPool1d,sampling_factor = 2,):
        super(TCNDec, self).__init__()
        """
        Parameters
        ----------
        ts_dimension : int
            The dimension of the time series (default is 1)
        dilations : tuple
            The dilation rates used in the TCN-AE model (default is (1, 2, 4, 8, 16))
        nb_filters : int
            The number of filters used in the dilated convolutional layers. All dilated conv. layers use the same number of filters (default is 20)
        """
        
        self.seq_len = seq_len
        self.ts_dimension = ts_dimension
        self.dilations = dilations
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.nb_stacks = nb_stacks
        self.padding = padding
        self.dropout_rate = dropout_rate
        self.filters_conv1d = filters_conv1d
        self.pooler = pooler
        self.sampling_factor = sampling_factor
        self.batch_size = batch_size
        
        # build the model
        self.build_model()
        
    def build_model(self):
        """Builds the TCN-AE model.
        If the argument `verbose` isn't passed in, the default verbosity level is used.
        Parameters
        ----------
        verbose : str, optional
            The verbosity level (default is 1)
            
        Returns
        -------
        KerasXYZType
        Todo
        Raises
        ------
        NotImplementedError
            If ...
        """
        self.input_conv1d = self.nb_filters[-1] if isinstance(self.nb_filters,list) else self.nb_filters

        # Put signal through TCN. Output-shape: (batch,sequence length, nb_filters)
        self.dec_upsample = nn.Upsample(scale_factor=self.sampling_factor)

        self.tcn_dec = TCN(8,nb_filters=self.nb_filters, kernel_size=self.kernel_size, nb_stacks=self.nb_stacks, dilations=self.dilations, 
                                padding=self.padding, use_skip_connections=True, dropout_rate=self.dropout_rate, return_sequences=True)
        
        self.dec_upscale_conv = nn.Conv1d(self.input_conv1d,self.seq_len, kernel_size=1, padding=self.padding)

        # Put the filter-outputs through a dense layer finally, to get the reconstructed signal
        self.linear_out = nn.Linear(self.ts_dimension, self.ts_dimension)
        
    def forward(self,x):
        x = x.view((self.batch_size, self.filters_conv1d, self.ts_dimension//self.sampling_factor))
        x = self.dec_upsample(x)
        x = self.tcn_dec(x)
        x = self.dec_upscale_conv(x)
        x = self.linear_out(x)
        
        return x
        
        
class TCNEncDec(nn.Module):
    """
    A class used to represent the Temporal Convolutional Autoencoder (TCN-AE).
    ...
    Attributes
    ----------
    model : xxtypexx
        The TCN-AE model.
    Methods
    -------
    build_model(verbose = 1)
        Builds the model
    """
    
    def __init__(self,seq_len = 50,ts_dimension = 1,batch_size = 32,dilations = (1, 2, 4, 8, 16),nb_filters = 20,kernel_size = 20,nb_stacks = 1,padding = 'same',dropout_rate = 0.00,filters_conv1d = 8,pooler = nn.AvgPool1d,sampling_factor = 2,):
        super(TCNEncDec, self).__init__()
        """
        Parameters
        ----------
        ts_dimension : int
            The dimension of the time series (default is 1)
        dilations : tuple
            The dilation rates used in the TCN-AE model (default is (1, 2, 4, 8, 16))
        nb_filters : int
            The number of filters used in the dilated convolutional layers. All dilated conv. layers use the same number of filters (default is 20)
        """
        
        self.seq_len = seq_len
        self.ts_dimension = ts_dimension
        self.dilations = dilations
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.nb_stacks = nb_stacks
        self.padding = padding
        self.dropout_rate = dropout_rate
        self.filters_conv1d = filters_conv1d
        self.pooler = pooler
        self.sampling_factor = sampling_factor
        self.batch_size = batch_size
        
        # build the model
        self.build_model()
        
    
    def build_model(self):
        """Builds the TCN-AE model.
        If the argument `verbose` isn't passed in, the default verbosity level is used.
        Parameters
        ----------
        verbose : str, optional
            The verbosity level (default is 1)
            
        Returns
        -------
        KerasXYZType
        Todo
        Raises
        ------
        NotImplementedError
            If ...
        """
        self.encoder = TCNEnc(seq_len=self.seq_len, ts_dimension=self.ts_dimension,dilations = self.dilations,nb_filters = self.nb_filters,kernel_size = self.kernel_size,nb_stacks = self.nb_stacks,padding = self.padding,dropout_rate = self.dropout_rate,filters_conv1d = self.filters_conv1d,pooler = self.pooler,sampling_factor = self.sampling_factor,batch_size = self.batch_size)
        
        self.decoder = TCNDec(seq_len = self.seq_len,ts_dimension = self.ts_dimension,dilations = self.dilations,nb_filters = self.nb_filters,kernel_size = self.kernel_size,nb_stacks = self.nb_stacks,padding = self.padding,dropout_rate = self.dropout_rate,filters_conv1d = self.filters_conv1d,pooler = self.pooler,sampling_factor = self.sampling_factor,batch_size = self.batch_size)
        
        
    def forward(self,x):
        embedding = self.encoder(x)
        x = self.decoder(embedding)
        return x, embedding


In [14]:
tcn_model = TCNEncDec(ts_dimension=36)

In [15]:
rand_sample = torch.randn((32,50,36))

In [16]:
rand_sample.shape 

torch.Size([32, 50, 36])

In [17]:
out = tcn_model(rand_sample)

/opt/anaconda3/envs/fyp/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1662707330697/work/aten/src/ATen/native/Convolution.cpp:894.)
  return F.conv1d(input, weight, bias, self.stride,


In [18]:
out[1].shape

torch.Size([32, 144])

In [19]:
out[0].shape

torch.Size([32, 50, 36])

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
def train_model(model, train_dataset, val_dataset, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    std_loss = nn.L1Loss(reduction='sum').to(device)
    #contrastive_loss = SupConLoss(contrast_mode="one").to(device)
    history = dict(train=[], val=[])
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0
  
    for epoch in range(1, n_epochs + 1):
        model = model.train()

        train_losses = []
        for in_seq,tar_seq,action in tqdm(train_dataset):
            optimizer.zero_grad()
            
            in_seq = in_seq.to(device)
            tar_seq = tar_seq.to(device)
            seq_pred, _ = model(in_seq)
            
            loss = std_loss(seq_pred, tar_seq)
            #loss += 0.5*contrastive_loss(embed,labels=sample_label.view(-1))
            #print(contrastive_loss(embed,labels=sample_label.view(-1)))

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for in_seq,tar_seq,action in val_dataset:

                in_seq = in_seq.to(device)
                tar_seq = tar_seq.to(device)
                seq_pred,_  = model(in_seq)

                loss = std_loss(seq_pred, tar_seq)
                #loss += 0.5*contrastive_loss(embed,labels=sample_label.view(-1))
                val_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        history['train'].append(train_loss)
        history['val'].append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')

    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [ ]:
model, history = train_model(
  tcn_model, 
  train_dl, 
  val_dl, 
  n_epochs=500
)

100%|██████████| 386/386 [01:01<00:00,  6.29it/s]


Epoch 1: train loss 14424.16729578206 val loss 11860.944478849085


100%|██████████| 386/386 [01:01<00:00,  6.32it/s]


In [ ]:
ax = plt.figure().gca()

ax.plot(history['train'])
ax.plot(history['val'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.title('Loss over training epochs')
plt.show();